## **Extract features from image using VGG**

### **Import necessary libraries**

In [1]:
import os 
import sys

import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt 
%matplotlib inline 
import seaborn as sns 

import warnings 
warnings.filterwarnings('ignore')

from PIL import Image 

import torch
import torchvision 
import torchvision.transforms as transforms 
import torchvision.models as models 
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn 
from torch.autograd import Variable
from Img2Vec import Img2Vec 


In [2]:
current_cwd = os.getcwd()
lst = current_cwd.split('\\', 2)
new_cwd = '/'.join(lst[:2])
os.chdir(new_cwd)
sys.path.append(new_cwd + '/code')

In [3]:
IMAGES_PATH = 'data/flickr8k/Images'
CAPTIONS_PATH = 'data/flickr8k/captions.txt'

data = pd.read_csv(CAPTIONS_PATH)
data

,image,caption
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...
...,...,...
40450,997722733_0cb5439472.jpg,A man in a pink shirt climbs a rock face
40451,997722733_0cb5439472.jpg,A man is rock climbing high in the air .
40452,997722733_0cb5439472.jpg,A person in a red shirt climbing up a rock fac...
40453,997722733_0cb5439472.jpg,A rock climber in a red shirt .


In [4]:
images_data = data.drop_duplicates(subset=['image'])
images_data

,image,caption
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...
5,1001773457_577c3a7d70.jpg,A black dog and a spotted dog are fighting
10,1002674143_1b742ab4b8.jpg,A little girl covered in paint sits in front o...
15,1003163366_44323f5815.jpg,A man lays on a bench while his dog sits by him .
20,1007129816_e794419615.jpg,A man in an orange hat starring at something .
...,...,...
40430,990890291_afc72be141.jpg,A man does a wheelie on his bicycle on the sid...
40435,99171998_7cc800ceef.jpg,A group is sitting around a snowy crevasse .
40440,99679241_adc853a5c0.jpg,A grey bird stands majestically on a beach whi...
40445,997338199_7343367d7f.jpg,A person stands near golden walls .


### **Load the VGG11BN**

In [5]:
vgg11_bn_image = {}
cnt = 0
img2vec = Img2Vec('vgg11bn')
for img in images_data.image:
    vgg11_bn_image[img] = img2vec.get_vector(IMAGES_PATH + '/' + img)
    cnt += 1
    if cnt % 1000 == 0:
        print(f"Completed {cnt} images")

Completed 1000 images
Completed 2000 images
Completed 3000 images
Completed 4000 images
Completed 5000 images
Completed 6000 images
Completed 7000 images
Completed 8000 images


In [6]:
vgg11_bn_features = images_data

In [7]:
vgg11_bn_features['image features'] = vgg11_bn_features.image.map(vgg11_bn_image)

In [8]:
vgg11_bn_features = vgg11_bn_features[['image', 'image features']].reindex()
with open('Image features/vgg11_bn.csv', mode='w', encoding='utf-8') as file: 
    vgg11_bn_features.to_csv(file, index=False)

### **Load the VGG13BN**

In [9]:
vgg13_bn_image = {}
cnt = 0
img2vec = Img2Vec('vgg13bn')
for img in images_data.image:
    vgg13_bn_image[img] = img2vec.get_vector(IMAGES_PATH + '/' + img)
    cnt += 1
    if cnt % 1000 == 0:
        print(f"Completed {cnt} images")

Completed 1000 images
Completed 2000 images
Completed 3000 images
Completed 4000 images
Completed 5000 images
Completed 6000 images
Completed 7000 images
Completed 8000 images


In [10]:
vgg13_bn_features = images_data

In [11]:
vgg13_bn_features['image features'] = vgg13_bn_features.image.map(vgg13_bn_image)

In [12]:
vgg13_bn_features = vgg13_bn_features[['image', 'image features']].reindex()
with open('Image features/vgg13_bn.csv', mode='w', encoding='utf-8') as file: 
    vgg13_bn_features.to_csv(file, index=False)

### **Load the VGG16BN**

In [13]:
vgg16_bn_image = {}
cnt = 0
img2vec = Img2Vec('vgg16bn')
for img in images_data.image:
    vgg16_bn_image[img] = img2vec.get_vector(IMAGES_PATH + '/' + img)
    cnt += 1
    if cnt % 1000 == 0:
        print(f"Completed {cnt} images")

Completed 1000 images
Completed 2000 images
Completed 3000 images
Completed 4000 images
Completed 5000 images
Completed 6000 images
Completed 7000 images
Completed 8000 images


In [14]:
vgg16_bn_features = images_data

In [15]:
vgg16_bn_features['image features'] = vgg16_bn_features.image.map(vgg16_bn_image)

In [16]:
vgg16_bn_features = vgg16_bn_features[['image', 'image features']].reindex()
with open('Image features/vgg16_bn.csv', mode='w', encoding='utf-8') as file: 
    vgg16_bn_features.to_csv(file, index=False)

### **Load the VGG19BN**

In [17]:
vgg19_bn_image = {}
cnt = 0
img2vec = Img2Vec('vgg19bn')
for img in images_data.image:
    vgg19_bn_image[img] = img2vec.get_vector(IMAGES_PATH + '/' + img)
    cnt += 1
    if cnt % 1000 == 0:
        print(f"Completed {cnt} images")

Completed 1000 images
Completed 2000 images
Completed 3000 images
Completed 4000 images
Completed 5000 images
Completed 6000 images
Completed 7000 images
Completed 8000 images


In [18]:
vgg19_bn_features = images_data

In [19]:
vgg19_bn_features['image features'] = vgg19_bn_features.image.map(vgg19_bn_image)

In [20]:
vgg19_bn_features = vgg19_bn_features[['image', 'image features']].reindex()
with open('Image features/vgg19_bn.csv', mode='w', encoding='utf-8') as file: 
    vgg19_bn_features.to_csv(file, index=False)